# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [54]:
from pyspark.sql import SparkSession

In [55]:
spark = SparkSession.builder.appName('cruise').getOrCreate()

In [56]:
from pyspark.ml.regression import LinearRegression

In [57]:
data = spark.read.csv('./data/cruise_ship_info.csv', header=True, inferSchema=True)

In [58]:
data.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [59]:
data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [60]:
for item in data.head(1)[0]:
    print(item)

Journey
Azamara
6
30.276999999999997
6.94
5.94
3.55
42.64
3.55


In [61]:
data.groupBy("Cruise_line").count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [62]:
from pyspark.ml.feature import StringIndexer

In [63]:
indexer = StringIndexer(inputCol='Cruise_line', outputCol='cruise_cat')
indexed = indexer.fit(data).transform(data)
indexed.head()

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0)

In [64]:
from pyspark.ml.feature import VectorAssembler

In [70]:
assembler = VectorAssembler(inputCols=[
     'Age',
     'Tonnage',
     'passengers',
     'length',
     'cabins',
     'passenger_density',
    'cruise_cat'
], outputCol='features')

In [72]:
output = assembler.transform(indexed)

In [73]:
output.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----------+--------------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|cruise_cat|            features|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----------+--------------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|      16.0|[6.0,30.276999999...|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|      16.0|[6.0,30.276999999...|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|       1.0|[26.0,47.262,14.8...|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|       1.0|[11.0,110.0,29.74...|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0| 

In [74]:
final_data = output.select('features', 'crew')

In [75]:
final_data.show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [76]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [77]:
train_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              108|
|   mean|8.240370370370378|
| stddev|3.457841670814657|
|    min|             0.59|
|    max|             21.0|
+-------+-----------------+



In [78]:
test_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|               50|
|   mean|6.830399999999998|
| stddev|3.439788769024953|
|    min|             0.59|
|    max|             13.6|
+-------+-----------------+



In [79]:
lr = LinearRegression(labelCol="crew")

In [80]:
lr_model = lr.fit(train_data)

In [81]:
test_results = lr_model.evaluate(test_data)

In [82]:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|  0.3324763318834041|
|  0.4670096303299349|
| -0.8588556844150137|
|-0.37309630960896456|
|  0.3994874864793072|
|-0.23035520721961156|
|  0.9789532152695468|
|-0.13621635203232785|
|-0.06360521783933848|
|-0.24953710798141948|
| 0.25093032776581303|
|-0.29323772258725533|
|-0.11899324322393667|
| 0.22122694370141893|
|-0.10893660132831329|
|  1.0188899496563586|
| 0.16162852170208597|
|-0.23314665105293653|
| 0.16327256122370493|
|  0.8306456352854674|
+--------------------+
only showing top 20 rows



In [83]:
test_results.meanSquaredError

0.6044618163780995

In [84]:
test_results.r2

0.9478710175324214

In [85]:
unlabeled_data = test_data.select("features")

In [86]:
predictions = lr_model.transform(unlabeled_data)

In [87]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[5.0,115.0,35.74,...|11.867523668116595|
|[5.0,122.0,28.5,1...| 6.232990369670065|
|[6.0,30.276999999...|4.4088556844150135|
|[6.0,112.0,38.0,9...|11.273096309608965|
|[6.0,113.0,37.82,...|11.600512513520693|
|[6.0,158.0,43.7,1...|13.830355207219611|
|[8.0,91.0,22.44,9...|10.021046784730453|
|[9.0,59.058,17.0,...| 7.536216352032328|
|[9.0,116.0,26.0,9...|11.063605217839338|
|[10.0,58.825,15.6...|7.2495371079814195|
|[10.0,77.0,20.16,...| 8.749069672234187|
|[10.0,81.76899999...| 8.713237722587255|
|[10.0,91.62700000...| 9.118993243223937|
|[11.0,58.6,15.66,...| 7.378773056298581|
|[11.0,86.0,21.24,...| 9.408936601328314|
|[11.0,108.977,26....|10.981110050343641|
|[12.0,2.329,0.94,...| 0.438371478297914|
|[12.0,25.0,3.88,5...|3.1031466510529366|
|[12.0,42.0,14.8,7...| 6.636727438776295|
|[12.0,91.0,20.32,...| 9.159354364714533|
+--------------------+------------

In [88]:
predictions.describe().show()

+-------+------------------+
|summary|        prediction|
+-------+------------------+
|  count|                50|
|   mean| 6.657073487714993|
| stddev| 3.189966928853478|
|    min| 0.438371478297914|
|    max|13.830355207219611|
+-------+------------------+



## Testing correlation

In [89]:
from pyspark.sql.functions import corr 

In [91]:
data.describe().show()

+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|summary|Ship_name|Cruise_line|               Age|           Tonnage|       passengers|           length|            cabins|passenger_density|             crew|
+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|  count|      158|        158|               158|               158|              158|              158|               158|              158|              158|
|   mean| Infinity|       null|15.689873417721518| 71.28467088607599|18.45740506329114|8.130632911392404| 8.830000000000005|39.90094936708861|7.794177215189873|
| stddev|     null|       null| 7.615691058751413|37.229540025907866|9.677094775143416|1.793473548054825|4.4714172221480615| 8.63921711391542|3.503486564627034|
|    min|Adventure|    Azamara|   

In [95]:
data.select(corr('crew','passengers')).show()

+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+

